In [9]:
import numpy as np
import pandas as pd

In [12]:
# marginal data p(x_1, x_2)
d1 = pd.DataFrame(np.random.randint(2, size=(100,2)), 
                       columns=['x_1', 'x_2']) 
print(d1.head())

# marginal data p(x_2, x_3) 
d2 = pd.DataFrame(np.random.randint(2, size=(100,2)), 
                       columns=['x_2', 'x_3']) 
print(d2.head())

   x_1  x_2
0    1    1
1    0    1
2    0    0
3    0    1
4    1    1
   x_2  x_3
0    0    1
1    0    0
2    1    0
3    1    0
4    0    0


In [33]:
p_a = {
    (0, 0): 0.1,
    (1, 0): 0.3,
    (0, 1): 0.4
}
p_b = {
    (0, 0): 0.5,
    (1, 0): 0.1,
    (0, 1): 0.2
}


In [38]:
# Define the matrix A (7x7)
A = np.array([
    [1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 0, 0],
    [0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 1, 0],
    [1, 0, 1, 0, 0, 0, 0]
])

# Define constants
c = 0.02  # Example value for c
p_a = {
    (0, 0): 0.2,
    (1, 0): 0.3,
    (0, 1): 0.4
}
p_b = {
    (0, 0): 0.25,
    (1, 0): 0.35,
    (0, 1): 0.15
}

# Define the right-hand side vector b
b = np.array([
    1 - c,
    p_a[(0, 0)] - c,
    p_a[(1, 0)],
    p_a[(0, 1)],
    p_b[(0, 0)] - c,
    p_b[(1, 0)],
    p_b[(0, 1)]
])

# Solve the system A * p = b
try:
    p = np.linalg.solve(A, b)
    print("Solution for [p_{001}, p_{100}, p_{101}, p_{010}, p_{011}, p_{110}, p_{111}]:")
    print(p)
except np.linalg.LinAlgError:
    print("The matrix is singular; no unique solution exists.")


The matrix is singular; no unique solution exists.


In [41]:
import numpy as np
from scipy.optimize import linprog

# Coefficients for the objective function (zero since we just want feasibility)
c = np.zeros(7)

# Coefficients for the inequality constraints (no inequalities in this problem)
A_ineq = None
b_ineq = None

# Coefficients for the equality constraints
A_eq = np.array([
    [1, 1, 1, 1, 1, 1, 1],  # p_{001} + p_{100} + p_{010} + p_{101} + p_{011} + p_{110} + p_{111} = 1 - c
    [1, 0, 0, 0, 0, 0, 0],  # p_{001} = \hat{p}_a(0, 0) - c
    [0, 1, 1, 0, 0, 0, 0],  # p_{100} + p_{101} = \hat{p}_a(1, 0)
    [0, 0, 0, 1, 1, 0, 0],  # p_{010} + p_{011} = \hat{p}_a(0, 1)
    [0, 1, 0, 0, 0, 0, 0],  # p_{100} = \hat{p}_b(0, 0) - c
    [0, 0, 0, 1, 0, 1, 0],  # p_{010} + p_{110} = \hat{p}_b(1, 0)
    [1, 0, 1, 0, 0, 0, 0]   # p_{001} + p_{101} = \hat{p}_b(0, 1)
])

# Define constants
c_val = 0.5  # Example value for c
p_a = {
    (0, 0): 0.2,
    (1, 0): 0.3,
    (0, 1): 0.4
}
p_b = {
    (0, 0): 0.25,
    (1, 0): 0.35,
    (0, 1): 0.15
}

# Define the right-hand side vector b (must include the adjusted value for c)
b_eq = np.array([
    1 - c_val,
    p_a[(0, 0)] - c_val,
    p_a[(1, 0)],
    p_a[(0, 1)],
    p_b[(0, 0)] - c_val,
    p_b[(1, 0)],
    p_b[(0, 1)]
])

# Bounds for each variable (0 <= p <= 1)
bounds = [(0, 1) for _ in range(7)]

# Solve the linear programming problem
result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds)

# Check the result
if result.success:
    print("Solution for [p_{001}, p_{100}, p_{101}, p_{010}, p_{011}, p_{110}, p_{111}]:")
    print(result.x)
else:
    print("The optimization did not converge:", result.message)


The optimization did not converge: The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is At lower/fixed bound)


In [45]:
import numpy as np
from scipy.optimize import linprog

# Coefficients for the objective function (zero since we just want feasibility)
c = np.zeros(7)

# Coefficients for the equality constraints
A_eq = np.array([
    [1, 1, 1, 1, 1, 1, 1],  # p_{001} + p_{100} + p_{010} + p_{101} + p_{011} + p_{110} + p_{111} = 1 - c
    [1, 0, 0, 0, 0, 0, 0],  # p_{001} = \hat{p}_a(0, 0) - c
    [0, 1, 1, 0, 0, 0, 0],  # p_{100} + p_{101} = \hat{p}_a(1, 0)
    [0, 0, 0, 1, 1, 0, 0],  # p_{010} + p_{011} = \hat{p}_a(0, 1)
    [0, 1, 0, 0, 0, 0, 0],  # p_{100} = \hat{p}_b(0, 0) - c
    [0, 0, 0, 1, 0, 1, 0],  # p_{010} + p_{110} = \hat{p}_b(1, 0)
    [1, 0, 1, 0, 0, 0, 0]   # p_{001} + p_{101} = \hat{p}_b(0, 1)
])

# Define constants
c_val = 0.199  # Example value for c
p_a = {
    (0, 0): 0.2,
    (1, 0): 0.3,
    (0, 1): 0.4
}
p_b = {
    (0, 0): 0.25,
    (1, 0): 0.35,
    (0, 1): 0.15
}

# Define the right-hand side vector b (must include the adjusted value for c)
b_eq = np.array([
    1 - c_val,                         # First equation: 1 - c
    p_a[(0, 0)] - c_val,               # Second equation: p_{001}
    p_a[(1, 0)],                       # Third equation: p_{100} + p_{101}
    p_a[(0, 1)],                       # Fourth equation: p_{010} + p_{011}
    p_b[(0, 0)] - c_val,               # Fifth equation: p_{100}
    p_b[(1, 0)],                       # Sixth equation: p_{010} + p_{110}
    p_b[(0, 1)]                        # Seventh equation: p_{001} + p_{101}
])

# Print the right-hand side vector to diagnose issues
print("Right-hand side vector (b_eq):", b_eq)

# Bounds for each variable (0 <= p <= 1)
bounds = [(0, 1) for _ in range(7)]

# Solve the linear programming problem
result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# Check the result
if result.success:
    print("Solution for [p_{001}, p_{100}, p_{101}, p_{010}, p_{011}, p_{110}, p_{111}]:")
    print(result.x)
else:
    print("The optimization did not converge:", result.message)


Right-hand side vector (b_eq): [0.801 0.001 0.3   0.4   0.051 0.35  0.15 ]
The optimization did not converge: The problem is infeasible. (HiGHS Status 8: model_status is Infeasible; primal_status is At lower/fixed bound)


In [34]:
# Define the matrix of coefficients (7x7 matrix)
A = np.array([
    [1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 0, 0],
    [0, 1, 1, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 0, 0],
    [0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 1, 0],
    [1, 0, 1, 0, 0, 0, 0]
])

print(np.linalg.det(A))

# Define the constants
c = 0.5  # Example value; set c according to your context

# Define the right-hand side vector
b = np.array([
    1 - c,
    p_a[(0, 0)] - c,
    p_a[(1, 0)],
    p_a[(0, 1)],
    p_b[(0, 0)] - c,
    p_b[(1, 0)],
    p_b[(0, 1)]
])

# Solve the linear system A * x = b
try:
    x = np.linalg.solve(A, b)
    print("Solution for [p_{001}, p_{100}, p_{101}, p_{010}, p_{011}, p_{110}, p_{111}]:")
    print(x)
except np.linalg.LinAlgError:
    print("The matrix is singular; no unique solution exists.")

0.0
The matrix is singular; no unique solution exists.
